# Heating Mesh Tally on CAD geometry made from Shapes

This constructs a reactor geometry from 3 Shape objects each made from points.

The Shapes made include a breeder blanket, PF coil and a central column shield.

2D and 3D Meshes tally are then simulated to show nuclear heating, flux and tritium_production across the model.

This makes a 3D geometry and material for the centre column

In [ ]:
import paramak

center_column = paramak.RotateMixedShape(
    points=[
        (50, 600, 'straight'),
        (150, 600, 'spline'),
        (100, 0, 'spline'),
        (150, -600, 'straight'),
        (50, -600, 'straight')
    ],
    name='center_column'
)

This makes a 3D geometry and material for breeder blanket. The azimuth_placement_angle argument is used to repeat the geometry around the Z axis at specified angles.

In [ ]:
blanket = paramak.RotateSplineShape(
    points=[
        (600, 0),
        (600, -20),
        (500, -300),
        (400, -300),
        (400, 0),
        (400, 300),
        (500, 300),
        (600, 20)
    ],
    name='blanket',
    rotation_angle=40,
    azimuth_placement_angle=[0, 45, 90, 135],
)

This makes a reactor object from the three components

In [ ]:
my_reactor = paramak.Reactor([blanket, center_column])

my_reactor.show()

This section forms the neutronics model by combining the 3D model, the plasma source and some assigned materials. Additionally, the tallies to record the heating are specified.

In [ ]:
my_reactor.export_dagmc_h5m(filename='dagmc.h5m')

This next section combines makes the materials, it also checks the material tags appear in the dagmc.h5m file

In [ ]:
# makes use of the previously created neutronics geometry (h5m file) and assigns
# actual materials to the material tags. 

import openmc

import neutronics_material_maker as nmm

# this links the material tags in the dagmc h5m file with materials.
# these materials are input as strings so they will be looked up in the
# neutronics material maker package

    
mat1 = nmm.Material.from_library(name='DT_plasma').openmc_material
mat1.name = 'blanket'
mat2 = nmm.Material.from_library(name='Li4SiO4').openmc_material
mat2.name = 'center_column'

materials = openmc.Materials([mat1, mat2]

This section makes the OpenMC geometry from a DAGMC file. This particular DAGMC file does not have a "graveyard" surface at the edge of the geometry so the ```bound_universe()``` methodgeometry = openmc.DAGMCUniverse(filename='dagmc.h5m')
 is used to automatically add the vacuum surface to the edge of the geometry

In [ ]:
bound_dag_univ = openmc.DAGMCUniverse(filename='dagmc.h5m').bounded_universe()
geometry = openmc.Geometry(root=bound_dag_univ)

This section sets the the neutronics results to record (know as tallies).

In [ ]:

mesh = openmc.RegularMesh()
mesh.dimension = [100, 100, 100]  # number of mesh cells in each dimension (resolution)

# finds bounding box size from the geometry size
mesh.lower_left = geometry.bounding_box[0] # x,y,z coordinates
mesh.upper_right = geometry.bounding_box[1] # x,y,z coordinates

mesh_filter = openmc.MeshFilter(mesh)

# Create flux mesh tally to score alpha production
mesh_tally = openmc.Tally(tally_id=1, name='alpha_production_on_mesh')  # note the tally_id is specified
mesh_tally.filters = [mesh_filter]
mesh_tally.scores = ['(n,Xa)']  # where X is a wild card


tallies = openmc.Tallies([mesh_tally])

Makes a simple ring source with a DT neutron energy spectrum

In [ ]:
import openmc_plasma_source as ops
# assigns a ring source of DT energy neutrons to the source using the
# openmc_plasma_source package, more details here
# https://github.com/fusion-energy/openmc-plasma-source/
my_source = ops.FusionRingSource(fuel="DT", radius=350)

Establishes the settings for the simulation.

In [ ]:
settings = openmc.Settings()
settings.inactive = 0
settings.run_mode = 'fixed source'
settings.batches = 4
settings.particles = 1000
settings.source = my_source

Sets simulation intensity, combines the geometry, materials, tallies and settings into a single object and runs the simulation

In [ ]:
my_model = openmc.Model(
    materials=materials,
    geometry=geometry,
    settings=settings,
    tallies=tallies
)
!rm *.h5  # just removes old summary.h5 or statepoint files
statepoint_file = my_model.run()

The following section opens the statepoint and converts the tally result into a VTK file. VTK files can be opened in paraview

In [ ]:

# assumes you have a statepoint file from the OpenMC simulation
statepoint = openmc.StatePoint('statepoint.4.h5')

# this shows the tallies present in the statepoint file
print(statepoint.tallies)

# loads up a tally from the statepoint using it's name
my_tally = statepoint.get_tally(name='alpha_production_on_mesh')

# converts the tally result into a VTK file
mesh.write_data_to_vtk(
    datasets={'mean': my_tally.mean},
    filename = "shape_alpha_production_on_mesh.vtk",
)

Now download the VTK file by moving to the file explorer, right mouse clicking on the file and selecting Download.

Open the file in Paraview or another VTK file viewer on your local computer